In [ ]:
!pip install pandas mysql-connector-python

### imports

In [4]:
import pandas as pd 
import mysql.connector

In [5]:
data = pd.read_csv('online_retail_II.csv', index_col=0)

In [6]:
data.head()

,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Invoice,,,,,,,
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 489434 to 581587
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   StockCode    1067371 non-null  object 
 1   Description  1062989 non-null  object 
 2   Quantity     1067371 non-null  int64  
 3   InvoiceDate  1067371 non-null  object 
 4   Price        1067371 non-null  float64
 5   Customer ID  824364 non-null   float64
 6   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 65.1+ MB


In [14]:
len(data)

1067371

In [15]:
data.dropna()

,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Invoice,,,,,,,
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...
581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [10]:
data1['customer_id'].unique()

array([13085., 13078., 15362., ..., 15520., 13298., 12713.], shape=(5881,))

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Read CSV
df = pd.read_csv("online_retail_II.csv")

# Clean data
df = df.dropna(subset=["Customer ID"]).query("Quantity > 0")
df = df.dropna()
df = df.drop_duplicates()
df["Customer ID"] = df["Customer ID"].astype(str)

# Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# Extract dimensions
products = df[["StockCode", "Description"]].drop_duplicates(subset=["StockCode"]).rename(columns={"StockCode": "stock_code", "Description": "description"})
customers = df[["Customer ID", "Country"]].drop_duplicates(subset=['Customer ID']).rename(columns={"Customer ID": "customer_id", "Country": "country"})
sales = df[["Invoice", "StockCode", "Customer ID", "Quantity", "Price", "InvoiceDate"]].rename(
    columns={"Invoice": "invoice_no", "StockCode": "stock_code", "Customer ID": "customer_id", "Quantity": "quantity",
             "Price": "unit_price", "InvoiceDate": "invoice_date"}
)

# Connect to local MySQL
try:
    conn = mysql.connector.connect(host="localhost", user="root", password="password", database="ecommerce")
    cursor = conn.cursor()
    print("Connection established")

    # Bulk insert products
    product_data = [(row["stock_code"], row["description"]) for _, row in products.iterrows()]
    cursor.executemany("INSERT INTO products (stock_code, description) VALUES (%s, %s)", product_data)
    conn.commit()
    print(f"Inserted {len(product_data)} products")

    # Bulk insert customers
    customer_data = [(row["customer_id"], row["country"]) for _, row in customers.iterrows()]
    cursor.executemany("INSERT INTO customers (customer_id, country) VALUES (%s, %s)", customer_data)
    conn.commit()
    print(f"Inserted {len(customer_data)} customers")

    # Bulk insert sales (in chunks to avoid timeouts)
    sales_data = [
        (row["invoice_no"], row["stock_code"], row["customer_id"], row["quantity"], row["unit_price"], row["invoice_date"])
        for _, row in sales.iterrows()
    ]
    batch_size = 10000
    for i in range(0, len(sales_data), batch_size):
        cursor.executemany(
            "INSERT INTO sales (invoice_no, stock_code, customer_id, quantity, unit_price, invoice_date) VALUES (%s, %s, %s, %s, %s, %s)",
            sales_data[i:i+batch_size]
        )
        conn.commit()
        print(f"Inserted {min(i+batch_size, len(sales_data))} sales rows")

    # Verify counts
    cursor.execute("SELECT COUNT(*) FROM products")
    print(f"Products in DB: {cursor.fetchone()[0]}")
    cursor.execute("SELECT COUNT(*) FROM customers")
    print(f"Customers in DB: {cursor.fetchone()[0]}")
    cursor.execute("SELECT COUNT(*) FROM sales")
    print(f"Sales in DB: {cursor.fetchone()[0]}")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()
    print("MySQL connection closed")

# Save CSVs for S3
products.to_csv("products.csv", index=False)
customers.to_csv("customers.csv", index=False)
sales.to_csv("sales.csv", index=False)
print("CSVs saved: products.csv, customers.csv, sales.csv")

Connection established
Inserted 10000 sales rows
Inserted 20000 sales rows
Inserted 30000 sales rows
Inserted 40000 sales rows
Inserted 50000 sales rows
Inserted 60000 sales rows
Inserted 70000 sales rows
Inserted 80000 sales rows
Inserted 90000 sales rows
Inserted 100000 sales rows
Inserted 110000 sales rows
Inserted 120000 sales rows
Inserted 130000 sales rows
Inserted 140000 sales rows
Inserted 150000 sales rows
Inserted 160000 sales rows
Inserted 170000 sales rows
Inserted 180000 sales rows
Inserted 190000 sales rows
Inserted 200000 sales rows
Inserted 210000 sales rows
Inserted 220000 sales rows
Inserted 230000 sales rows
Inserted 240000 sales rows
Inserted 250000 sales rows
Inserted 260000 sales rows
Inserted 270000 sales rows
Inserted 280000 sales rows
Inserted 290000 sales rows
Inserted 300000 sales rows
Inserted 310000 sales rows
Inserted 320000 sales rows
Inserted 330000 sales rows
Inserted 340000 sales rows
Inserted 350000 sales rows
Inserted 360000 sales rows
Inserted 37000

In [50]:
products = df[["StockCode", "Description"]].drop_duplicates().rename(columns={"StockCode": "stock_code", "Description": "description"})

In [60]:
len(products)

4631

In [53]:
data = products.drop_duplicates()

In [58]:
data[['stock_code']].drop_duplicates(keep="first")
len(data)

5315

In [2]:
import pandas as pd

data1 = pd.read_csv('sales.csv')
data2 = pd.read_csv('products.csv')
data3 = pd.read_csv('customers.csv')

print('sales columns = ', data1.columns)
print('product columns = ', data2.columns)
print('customers columns = ', data3.columns)

sales columns =  Index(['invoice_no', 'stock_code', 'customer_id', 'quantity', 'unit_price',
       'invoice_date'],
      dtype='object')
product columns =  Index(['stock_code', 'description'], dtype='object')
customers columns =  Index(['customer_id', 'country'], dtype='object')
